In [ ]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Importing Necessary Libraries**

In [ ]:
import pandas as pd
import numpy as np
import re
import nlpaug.augmenter.word as naw
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

**Loading the Dataset**

In [ ]:
#reading the dataset in csv format

data = pd.read_csv('/content/drive/MyDrive/Datasets/gittercom_annotated_data.csv')

**Text Preprocessing**


In [ ]:
def text_preprocessing(text):
    #removing punctuation from text data
    text = re.sub(r'[^\w\s]', '', text)

    #removing special symbols from text data
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    #replacing usernames in text data with '<username>'
    text = re.sub(r'@[^\s]+', '<username>', text)

    #replacing url in text data with '<url>' 
    text = re.sub(r'http\S+|www\S+|\S+\.com\S+', '<url>', text)
    
    #replacing code snippets in text data with '<code>' 
    text = re.sub(r'`[^`]+`', '<code>', text)
    
    return text

**Applying Text Preprocessing to the Text Data**

In [ ]:
#applying preprocessing techniques to the text data ('message' column holds all textual communications,i.e., text data)

data['message'] = data['message'].apply(text_preprocessing)

**Counting the Total Number of Instances of Our Dataset**

In [ ]:
data.shape[0]

1000

In [ ]:
#exploring first five rows of our dataset

data.head()

,Channel,message,category
0,Cucumber,Hi Team I just recently upgraded our cucumberj...,fear
1,Cucumber,So github is trying to replace irc P,fear
2,Cucumber,aslakhellesoy Thanks seems like I was using o...,joy
3,Cucumber,Sidkiyassine just call the methods directly,surprise
4,Cucumber,Hello guys need a helpI want to call the run m...,fear


In [ ]:
#exploring the instances of message column that holds text data

data['message']

0      Hi Team I just recently upgraded our cucumberj...
1                   So github is trying to replace irc P
2      aslakhellesoy Thanks  seems like I was using o...
3            Sidkiyassine just call the methods directly
4      Hello guys need a helpI want to call the run m...
                             ...                        
995    Besides the unzip issue which I saw you solved...
996    Hopefully now that the unzip issue is fixed we...
997     I can see that being a problem with my community
998                       which would slow down installs
999         Thanks yet again Ill try your suggestion now
Name: message, Length: 1000, dtype: object

**Splitting the Dataset for Classification**

In [ ]:
X = data['message'].values
y = data['category'].values

#splitting the data into training and testing sets (70% for training and 30% for testing)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Data Augmentation of training data using Synonym**

In [ ]:
augmented_data = []
augmented_category = []
no_of_targeted_augmentation = 30000

#initializing SynonymAug class of nlp aug library that leverage semantic meaning to substitute word using synonym
data_augmentation = naw.SynonymAug(aug_src='wordnet')

while len(augmented_data) < no_of_targeted_augmentation:
    for message, category in zip(X_train, y_train):
        augmented_text = data_augmentation.augment(message)
        augmented_data.append(augmented_text)
        augmented_category.append(category)
        if len(augmented_data) == no_of_targeted_augmentation:
            break

In [ ]:
X_train = X_train.tolist() + augmented_data[:no_of_targeted_augmentation]
y_train = y_train.tolist() + augmented_category[:no_of_targeted_augmentation]

**Counting the Total Number of Instances of training data After Data Augmentation**

In [ ]:
len(X_train)

30700

**Dropping Any Possible Duplicate Instance(s) of Text Data Due to Performing Data Augmentation**

In [ ]:
data.drop_duplicates(subset='message', inplace=True)

In [ ]:
len(X_train)

30700

**Exploring Augmented Data**

In [ ]:
for message, category in zip(X_train, y_train):
    print("message:", message)
    print("category:", category)

Streaming output truncated to the last 5000 lines.
message: ['also jdubois cannot regain the tips folder <url >']
category: sadness
message: ['howdy iodine am running into cucumberspring124 phoebe springboot130 issues <url> javalangArrayStoreException sunreflectannotationTypeNotPresentExceptionProxy']
category: fear
message: ['1 wish ace could do this kind of work for spare as well but unfortunately information technology not possible right now but information technology would be a pipe dream semen truthful to work full time on a project like this']
category: sadness
message: ['So Iodin would scarce extend that a litle bit perhaps with a few notes to that effect and everything would exist all effective']
category: joy
message: ['Tips and tricks expect awful']
category: joy
message: ['Ping ping pink Iu2019d like to let the jspm work flow for infamous published tonight D Just 380 needs a fix or is information technology my fault']
category: joy
message: ['Ic an finis this with this']
cat

**Converting Augmented Data Back into Strings for Feature Extraction**

In [ ]:
X_train = [' '.join(x) for x in X_train]

**Preparing the Data Before Feeding It to the TextCNN Model**

In [ ]:
#converting text data to sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

max_length = 100  

#padding the sequences
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')


**Performing One-Hot Encoding**

In [ ]:
one_hot_encoder = OneHotEncoder()
y_train = one_hot_encoder.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_test = one_hot_encoder.transform(np.array(y_test).reshape(-1, 1)).toarray()


**Building TextCNN Model**

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100  
filters = 128  
kernel_size = 5  

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(Conv1D(filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(6, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          688400    
                                                                 
 conv1d_2 (Conv1D)           (None, 96, 128)           64128     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 6)                 390       
                                                      

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Training the Model**

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64)

Epoch 1/50
480/480 [==============================] - 45s 91ms/step - loss: 0.5697 - accuracy: 0.7938 - val_loss: 2.4640 - val_accuracy: 0.4700
Epoch 2/50
480/480 [==============================] - 10s 22ms/step - loss: 0.0621 - accuracy: 0.9824 - val_loss: 3.0453 - val_accuracy: 0.4533
Epoch 3/50
480/480 [==============================] - 6s 12ms/step - loss: 0.0382 - accuracy: 0.9871 - val_loss: 3.2939 - val_accuracy: 0.4400
Epoch 4/50
480/480 [==============================] - 4s 9ms/step - loss: 0.0313 - accuracy: 0.9890 - val_loss: 4.0132 - val_accuracy: 0.4200
Epoch 5/50
480/480 [==============================] - 5s 11ms/step - loss: 0.0275 - accuracy: 0.9904 - val_loss: 4.3876 - val_accuracy: 0.4233
Epoch 6/50
480/480 [==============================] - 4s 8ms/step - loss: 0.0237 - accuracy: 0.9921 - val_loss: 4.5493 - val_accuracy: 0.4333
Epoch 7/50
480/480 [==============================] - 5s 10ms/step - loss: 0.0215 - accuracy: 0.9929 - val_loss: 4.8764 - val_accuracy: 0.4367

**Testing and Evaluating the Model**

In [ ]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)

#Generating Classification Report of the Model for Computing Precision, Recall and F1-Score for Every Six Emotion Categories
clsf_rprt = classification_report(np.argmax(y_test, axis=1), y_pred_labels)
print("Classification_report:\n", clsf_rprt)

10/10 [==============================] - 0s 2ms/step
Classification_report:
               precision    recall  f1-score   support

           0       0.29      0.18      0.22        28
           1       0.43      0.51      0.47        78
           2       0.50      0.56      0.53        88
           3       0.28      0.56      0.37         9
           4       0.45      0.36      0.40        88
           5       0.00      0.00      0.00         9

    accuracy                           0.44       300
   macro avg       0.32      0.36      0.33       300
weighted avg       0.43      0.44      0.43       300



In [ ]:
#calculating precision score
precision = precision_score(np.argmax(y_test, axis=1), y_pred_labels, average='weighted')

#calculating recall score
recall = recall_score(np.argmax(y_test, axis=1), y_pred_labels, average='weighted')

#calculating f1-score score
f1 = f1_score(np.argmax(y_test, axis=1), y_pred_labels, average='weighted')

#calculating accuracy score
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_labels)

print("Precision (weighted):", precision)
print("Recall (weighted):", recall)
print("F1-Score (weighted):", f1)
print("Accuracy:", accuracy)

Precision (weighted): 0.42529585103445033
Recall (weighted): 0.43666666666666665
F1-Score (weighted): 0.4254053345621521
Accuracy: 0.43666666666666665
